<div class="alert alert-block alert-warning">
<b>Warning:</b> Before running below cells please make sure you have API key. 
                Please see <a href="https://github.com/heremaps/here-map-widget-for-jupyter/blob/master/examples/README.md">README.md</a> for more info on API key.
</div>

In [ ]:
import os

os.environ["LS_API_KEY"] = "MY-API-KEY"  # replace your API key here.

In [ ]:
import json
import pandas as pd
from ipywidgets import link, FloatSlider
from branca.colormap import linear
import random

geo_json_data = json.load(open("us-states.json"))
unemployment = pd.read_csv("us_unemployment_oct2012.csv")

In [ ]:
unemployment = dict(zip(unemployment["State"].tolist(), unemployment["Unemployment"].tolist()))

In [ ]:
unemployment

In [ ]:
from here_map_widget import Map, Choropleth
import os

layer = Choropleth(
    geo_data=geo_json_data,
    choro_data=unemployment,
    colormap=linear.YlOrRd_04,
    style={"fillOpacity": 0.8, "dashArray": "5, 5"},
    disable_legacy_mode=True,
)

m = Map(api_key=os.environ["LS_API_KEY"])
m.center = [39.96214343725025, -96.72875165247453]
m.zoom = 3.556073061219927

In [ ]:
m.add_layer(layer)
m

In [ ]:
def compute_style(feature, colormap, choro_data):
    return {"fillColor": colormap.rgb_hex_str(choro_data), "strokeColor": "white"}

In [ ]:
layer.style_callback = compute_style

In [ ]:
from here_map_widget import WidgetControl

slider = FloatSlider(min=layer.value_min, max=layer.value_max, continuous_update=False)
slider.value = layer.value_min
link((slider, "value"), (layer, "value_min"))
wc = WidgetControl(widget=slider, alignment="BOTTOM_RIGHT")
m.add_control(wc)
m

In [ ]:
layer.colormap = linear.RdBu_11

In [ ]:
from ipywidgets import Text, HTML
from here_map_widget import WidgetControl, GeoJSON, FullscreenControl


html = HTML(
    """
    <h4> Name </h4>
    Hover over a state
"""
)
html.layout.margin = "0px 10px 10px 10px"
control = WidgetControl(widget=html, alignment="TOP_LEFT", name="HTML")
m.add_control(control)


def update_html(feature, id, **kwargs):
    html.value = """
        State name:  
        <b>{}\n</b>
        {} 
    """.format(
        id, feature["properties"]["name"]
    )


layer.on_hover(update_html)
m.add_control(FullscreenControl(alignment="TOP_RIGHT"))